This notebook is for snapping of substations to lines

In [101]:
import os
import matplotlib.pyplot as plt 
import geopandas as gpd
import geoplot
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from shapely.geometry import Point

# Africa shape data

In [102]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
africa = world.query('continent == "Africa"')
nigeriaplot = world.query('name == "Nigeria"')
#world

# Load data

In [103]:
# pathg = os.path.realpath("data")+'/africa_all_generators.geojson' # Generators are not required in base_network
paths = os.path.realpath("data/clean")+'/africa_all_buses_clean.geojson'
pathl = os.path.realpath("data/clean")+'/africa_all_lines_clean.geojson'

# generators = gpd.read_file(pathg).set_crs(epsg=4326, inplace=True)
substations = gpd.read_file(paths).set_crs(epsg=4326, inplace=True)
lines = gpd.read_file(pathl).set_crs(epsg=4326, inplace=True)

In [104]:
lines.head(2)

,line_id,bus0,bus1,voltage,circuits,length,underground,under_construction,tag_type,tag_frequency,country,geometry
0,380986751-1,None,None,220000,1,40467.295973,False,False,line,50,algeria,"LINESTRING (-1.83638 34.78691, -1.83590 34.789..."
1,173682341-1,None,None,400000,1,168.067783,False,False,line,50,algeria,"LINESTRING (-0.90176 35.08983, -0.90181 35.090..."


In [120]:
lines["bounds"] = lines["geometry"].boundary # create start and end point
# splits into coordinates
lines["bus0_lat"] = lines["bounds"].bounds.iloc[:,0]
lines["bus0_lon"] = lines["bounds"].bounds.iloc[:,1]
lines["bus1_lat"] = lines["bounds"].bounds.iloc[:,2]
lines["bus1_lon"] = lines["bounds"].bounds.iloc[:,3]

# TODO: Create Point(x,y) for each bus